In [1]:
import __init__
from tensorflow.keras.utils import Sequence
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from tensorflow.keras.layers import Concatenate,Embedding ,Dense ,Input,LSTM,Permute,Softmax,Lambda,Flatten,GRU,Dropout,BatchNormalization, Normalization, Attention, Bidirectional, Masking, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras_self_attention import SeqSelfAttention
from tensorflow.keras import Model
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
print("Num GPUs Available: ", tf.config.list_physical_devices('XPU'))

Num GPUs Available:  []


In [3]:
root_path = '../../..'
period = 'max'
interval = '1d'

class DataGenerator(Sequence):

    def __init__(self, meta_df, batch_size=64, time_series=30, input_shape=(20,13), output_shape=(20,5), partition_factor=1,**kwargs):
        super().__init__(**kwargs)
        self.ds_pointer = 0
        self.batch_size = batch_size
        self.meta_df = meta_df
        self.time_series = time_series
        self.partition_factor = partition_factor
        self.data = self.load_data()
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.input_buffer = np.zeros((batch_size,input_shape[0],input_shape[1]))
        self.label_buffer = np.zeros((batch_size,output_shape[0],output_shape[1]))
        self.reset_pointer()

    def load_data(self):
        #self.meta_df['Label'] = self.meta_df['Label'].astype(int)
        symbol = self.meta_df['Symbol'].unique()
        temp = {}
        for s in tqdm(symbol):
           temp[s] = pd.read_csv(f'{root_path}/ouput/analysis/price_{period}_{interval}/data/{s}.csv')
        return temp

    def reset_pointer(self, seed = 1314):
        self.meta_df = self.meta_df.sample(frac=1, random_state=seed)
        self.ds_pointer = 0

    def __len__(self):
        return len(self.meta_df)// self.batch_size// self.partition_factor

    def __getitem__(self, index):
        if self.ds_pointer + self.batch_size >= len(self.meta_df):
            self.reset_pointer()
            return self.__getitem__(index)
        to_get = self.meta_df.iloc[self.ds_pointer:self.ds_pointer+self.batch_size]
        to_get_symbol = to_get['Symbol']
        to_get_index = to_get['Index']

        for i, (index, symbol) in enumerate(zip(to_get_index, to_get_symbol)):
          self.input_buffer[i, :, :] = self.data[symbol].iloc[index-self.input_shape[0]:index,:]
          self.label_buffer[i, :, :-1] = self.data[symbol][['Open','High','Low','Close']].iloc[index:index+self.output_shape[0],:]

        # Use numpy to find highest high and lowest low
        high_values = self.label_buffer[:, :, 1]  # High column
        low_values = self.label_buffer[:, :, 2]   # Low column

        highest_high_indices = np.argmax(high_values, axis=1)
        lowest_low_indices = np.argmin(low_values, axis=1)

        # Last Column: Indicates the highest high and lowest low
        self.label_buffer[:, :, -1] = 0  # Initialize the last column to 0
        np.put_along_axis(self.label_buffer[:, :, -1], highest_high_indices[:, None], 1, axis=1)
        np.put_along_axis(self.label_buffer[:, :, -1], lowest_low_indices[:, None], -1, axis=1)

        batch_x = self.input_buffer
        batch_y = self.label_buffer

        self.ds_pointer+=self.batch_size

        return batch_x, batch_y

In [5]:
batch_size = 256


input_shape = (20,19)
meta_path = f'{root_path}/ouput/analysis/price_{period}_{interval}/meta.csv'
meta_df = pd.read_csv(meta_path)

# train_meta, test_meta = train_test_split(meta_df, test_size=0.2)
# train_meta.to_csv(f'{root_path}/ouput/analysis/price_{period}_{interval}/train_meta.csv',index=False)
# test_meta.to_csv(f'{root_path}/ouput/analysis/price_{period}_{interval}/test_meta.csv',index=False)

train_meta = pd.read_csv(f'{root_path}/ouput/analysis/price_{period}_{interval}/train_meta.csv')
test_meta = pd.read_csv(f'{root_path}/ouput/analysis/price_{period}_{interval}/test_meta.csv')

In [6]:
train_data = DataGenerator(train_meta, batch_size=batch_size, input_shape=input_shape, partition_factor=1)
test_data = DataGenerator(test_meta, batch_size=batch_size, input_shape=input_shape, partition_factor=100)

  0%|          | 0/502 [00:00<?, ?it/s]

  0%|          | 0/502 [00:00<?, ?it/s]

In [7]:
inp, oup = test_data.__getitem__(0)
print(inp[0][-1])
print(oup[0][:, 4])

[-0.22005437 -0.21878429 -0.21892754 -0.21969123 -0.48144629  0.
  0.          0.          0.          0.         -0.33299915 -0.43346996
  0.8445955   0.06907324 -0.11708631  0.          0.          0.
  0.        ]
[ 0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.]


In [13]:
@register_keras_serializable()
def r2_score(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

@register_keras_serializable()
def mda(y_true, y_pred):
    # Convert to float32
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    
    # Calculate the direction of the true and predicted values
    direction_true = K.sign(y_true[1:] - y_true[:-1])
    direction_pred = K.sign(y_pred[1:] - y_pred[:-1])
    
    # Calculate the Mean Directional Accuracy
    correct_directions = K.equal(direction_true, direction_pred)
    mda_value = K.mean(K.cast(correct_directions, 'float32'))
    
    return mda_value

@register_keras_serializable()
def custom_mae_loss(y_true, y_pred):
    # Convert to float32
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    
    # Calculate the absolute difference
    abs_diff = K.abs(y_true - y_pred)
    
    # Calculate the mean of the absolute differences
    mae_value = K.mean(abs_diff)
    
    return mae_value

def build_model(time_series,num_of_features, init_neuron=256):
    K.clear_session()
    input_ = Input(shape=(time_series,num_of_features), name='Input')
    input_dense = Dense(init_neuron)(input_)
    # Define GRU layer
    encoder = Bidirectional(GRU(init_neuron, return_state=True, return_sequences=True, name='Encoder'))
    encoder_outputs, forward_h, backward_h = encoder(input_dense)

    state_h = Concatenate()([forward_h, backward_h])
    # # Define attention layer
    attention = SeqSelfAttention()(encoder_outputs)

    # # Concatenate context vector and encoder outputs
    # concat_layer = Concatenate(axis=-1, name='Concatenate')
    # decoder_combined_context = concat_layer([attention, input_dense])

    # Define decoder
    decoder_gru = GRU(init_neuron*2, return_sequences=True, name='Decoder')

    # Pass the concatenated input through the decoder
    decoder_output = decoder_gru(attention, initial_state=state_h)
    output = TimeDistributed(Dense(5))(decoder_output)



    model = Model(inputs = input_ , outputs = output)
    adam_optimizer = Adam(learning_rate=0.0001, clipnorm=1., weight_decay=1e-7)
    model.compile(loss=custom_mae_loss,optimizer=adam_optimizer,metrics=[r2_score, mda])
    model.summary()
    return model

model = build_model(input_shape[0],input_shape[1])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 20, 19)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 20, 256)   │      5,120 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, 20, 512), │    789,504 │ dense[0][0]       │
│ (Bidirectional)     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ seq_self_attention  │ (None, 20, 512)   │     32,833 │ bidirectional[0]… │
│ (SeqSelfAttention)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder (GRU)       │ (None, 20, 512)   │  1,575,936 │ seq_self_attenti… │
│                     │                   │            │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 20, 5)     │      2,565 │ Decoder[0][0]     │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,405,958 (9.18 MB)

 Trainable params: 2,405,958 (9.18 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Directory Config
neural_path = 'neural_network'
log_path = 'neural_network/log'
save_path_dir='attention_t20_encode_decode'

if not os.path.exists(log_path):
    os.makedirs(log_path)
checkpoint = ModelCheckpoint(
    f'{neural_path}/{save_path_dir}/saved_model.keras',
    verbose=1,
    monitor='val_r2_score',
    save_weights_only=False,
    save_best_only=True,
    mode='max')

earlystop = EarlyStopping(
    monitor="val_r2_score",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=False
)
!mkdir neural_network

tensorboard = TensorBoard(log_dir=log_path)

�l�ؿ����ɮ� neural_network �w�g�s�b�C


In [10]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

AttributeError: module 'pydot' has no attribute 'InvocationException'

In [14]:
history = model.fit(train_data, batch_size=batch_size, epochs=2000, validation_data=test_data, callbacks=[checkpoint, earlystop, tensorboard])
plt.plot(history.history['r2_score'])
plt.plot(history.history['val_r2_score'])
plt.title('model R2')
plt.ylabel('R2')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print(f"Highest Val R2: {max(history.history['val_r2_score'])}")

Epoch 1/2000
  150/13114 ━━━━━━━━━━━━━━━━━━━━ 1:11:20 330ms/step - loss: 0.5820 - mda: 0.4591 - r2_score: -0.0656

KeyboardInterrupt: 

In [172]:
from tensorflow.keras.models import load_model
model = load_model(f'{neural_path}/{save_path_dir}/saved_model_epoch4.keras')

In [48]:
test_x, test_y = test_data.__getitem__(0)
predict_result = model.predict(test_x)
print(f"Actual: Max: {max(test_y[0][:,1])}, Min: {min(test_y[0][:,2])}, ArgMax: {np.argmax(test_y[0][:,-1])}, ArgMin: {np.argmin(test_y[0][:,-1])}")
print(f"Predict: Max: {max(predict_result[0][:,1])}, Min:{min(predict_result[0][:,2])}, ArgMax: {np.argmax(predict_result[0][:,-1])}, ArgMin: {np.argmin(predict_result[0][:,-1])}")
print(f"Model feed: Max: {max(test_x[0][:,1])}, Min: {min(test_x[0][:,2])}")
print(test_y[0][:,2].shape)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Actual: Max: 0.9679888273830812, Min: 0.9062574765036792, ArgMax: 10, ArgMin: 17
Predict: Max: 0.522071897983551, Min:0.4985039234161377, ArgMax: 19, ArgMin: 0
Model feed: Max: 0.925346474934392, Min: 0.8341400514156476
(20,)


In [194]:
#Transfer Learning
symbol = "MMM"
meta_path = f'{root_path}/ouput/analysis/price_{period}_{interval}/meta.csv'
meta_df = pd.read_csv(meta_path)
meta_df = meta_df[meta_df['Symbol']==symbol]

train_meta, test_meta = train_test_split(meta_df, test_size=0.2)
train_data = DataGenerator(train_meta, batch_size=batch_size, input_shape=input_shape, partition_factor=1)
test_data = DataGenerator(test_meta, batch_size=batch_size, input_shape=input_shape, partition_factor=1)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [197]:
# Directory Config
neural_path = 'neural_network'
log_path = 'neural_network/log'
save_path_dir='neural_network_transfer'

if not os.path.exists(log_path):
    os.makedirs(log_path)
checkpoint = ModelCheckpoint(
    f'{neural_path}/{save_path_dir}/saved_model_{symbol}.keras',
    verbose=1,
    monitor='val_r2_score',
    save_weights_only=False,
    save_best_only=True,
    mode='max')

earlystop = EarlyStopping(
    monitor="val_r2_score",
    min_delta=0,
    patience=20,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=False
)
!mkdir neural_network

tensorboard = TensorBoard(log_dir=log_path)


history = model.fit(train_data, batch_size=batch_size, epochs=2000, validation_data=test_data, callbacks=[checkpoint, earlystop, tensorboard])
plt.plot(history.history['r2_score'])
plt.plot(history.history['val_r2_score'])
plt.title('model R2')
plt.ylabel('R2')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print(f"Highest Val R2: {max(history.history['val_r2_score'])}")

�l�ؿ����ɮ� neural_network �w�g�s�b�C


Epoch 1/2000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - loss: 0.0026 - r2_score: 0.9976
Epoch 1: val_r2_score improved from -inf to 0.99764, saving model to neural_network/neural_network_transfer/saved_model_MMM.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 18s 350ms/step - loss: 0.0026 - r2_score: 0.9976 - val_loss: 0.0024 - val_r2_score: 0.9976
Epoch 2/2000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step - loss: 0.0023 - r2_score: 0.9978
Epoch 2: val_r2_score did not improve from 0.99764
48/48 ━━━━━━━━━━━━━━━━━━━━ 22s 447ms/step - loss: 0.0023 - r2_score: 0.9978 - val_loss: 0.0028 - val_r2_score: 0.9973
Epoch 3/2000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step - loss: 0.0022 - r2_score: 0.9979
Epoch 3: val_r2_score improved from 0.99764 to 0.99774, saving model to neural_network/neural_network_transfer/saved_model_MMM.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 30s 601ms/step - loss: 0.0022 - r2_score: 0.9979 - val_loss: 0.0024 - val_r2_score: 0.9977
Epoch 4/2000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step - loss: 0.0024

KeyboardInterrupt: 